In [2]:
import folium
import pandas as pd
import pickle
import os
import h3
from label_samples_time_hexa import label_samples
from vizualize import *
from func import *

In [3]:
matrix = pd.read_pickle("/home/jaro/BINP29/Project_Eran/1_dist_matrix/eucl_dist.pkl")

In [4]:
time_bins = 11
resolution = 3
same_age_range = True
df = label_samples("/home/jaro/BINP29/Project_Eran/", time_bins, resolution, same_age_range)
time_bins = rename_time_bins(df)
time_bins_hexagons = get_time_bin_hexagons(df)

In [32]:
# function that calculates the average distance between two groups of samples
def calc_avg_dist(samples_hex1, samples_hex2, dist_matrix):
    return dist_matrix.loc[samples_hex1, samples_hex2].values.flatten().mean()


def calc_neighbor_dist(hexagons, dist_matrix, time_bin_df, hex_col):
    # get the samples in each hexagon
    samples_in_hex = time_bin_df.groupby(hex_col)['ID'].apply(list).to_dict()
    # create a list of all values in samples_in_hex
    all_samples = [sample for samples in samples_in_hex.values() for sample in samples]
    # create a submatrix of the distance matrix for the samples in the hexagons
    dist_matrix = dist_matrix.loc[all_samples, all_samples]
    # initialize the dictionary to store the average distances between neighboring hexagons
    averages = {}
    
    # function which gets the neighbors for all hexagons usin Delauny triangulation
    def get_neighbors(hexagons):
        # get the centroid of every hexagon
        coord = []
        for hex in hexagons:
            coord.append(h3.h3_to_geo(hex))
        # change coord to and hexagons to np array
        coord = np.array(coord)
        hexagons = np.array(hexagons)
        # calc the Delaunay triangle
        tri = Delaunay(coord)
        # function to get the edges
        def tris2edges(tris):
            edges = set([])
            for tri in tris:
                for k in range(3):
                    i,j = tri[k], tri[(k+1)%3]
                    i,j = min(i,j), max(i,j)  # canonical edge representation
                    edges.add((i,j))
            return edges
        # get the edges (pairs of hexagons)
        edges = tris2edges(tri.simplices)
        # use the edges to create a dictionary with all the neighborhoods that we want to calculate
        neighbors = {}
        for (i,j) in edges:
            if hexagons[i] in neighbors:
                neighbors[hexagons[i]].append(hexagons[j])
            else:
                neighbors[hexagons[i]] = [hexagons[j]]
        return neighbors
    
    # get the neighbors for whom we will calculate the distances
    neighbors = get_neighbors(hexagons)
                
    # calculate the average distance between the hexagon and its neighbors
    for hexagon in neighbors.keys():
        neighbors[hexagon].append(hexagon)
        for neighbor in neighbors[hexagon]:
            Ids_in_hexagon = samples_in_hex.get(hexagon, [])
            Ids_in_neighbor = samples_in_hex.get(neighbor, [])
            # get the pair of hexagons
            pair = frozenset([hexagon, neighbor])

            # calculate the average distance between the hexagon and its neighbor
            distance = calc_avg_dist(Ids_in_hexagon, Ids_in_neighbor, dist_matrix)

            averages[pair] = round(distance, 2)

    return averages


# this function calculates the average distance between the each hexagon and its neighbors for each time bin
def calc_dist_time_bin(df, dist_matrix=None):
    
    # get column name for the hexagons (it should be the only column with 'hex' in the name)
    hex_col = str(df.columns[df.columns.str.contains('hex')][0])
    
    # Convert the 'AgeGroup' column values to tuples of integers representing the start and end years,
    df['AgeGroupTuple'] = df['AgeGroup'].apply(lambda x: tuple(map(int, x.split('-'))))
    
    # Sort the unique age group tuples to process them in a chronological order.
    time_bins = sorted(df['AgeGroupTuple'].unique())
    averages = {}
    # Iterate over each time bin.
    for time_bin in time_bins:
        # Format the current time bin as a string for labeling purposes.
        bin_label = rename_times(time_bin)
        
        # get subset of the data frame for that time bin
        time_bin_df = df[df['AgeGroupTuple'] == time_bin]

        # get all unique hexagons for that time bin
        hexagons = time_bin_df[hex_col].unique()
        
        # Calculate the average distance for each hexagon to its neighbors within the current time bin.
        average_distances = calc_neighbor_dist(hexagons, dist_matrix, time_bin_df, hex_col)

        # Append the calculated average distances to the dictionary, using the time bin label as the key.
        averages.update({bin_label: average_distances})

    # Return the dictionary with the average distances between neighboring hexagons for each time bin.
    return averages

def get_hexagons(time_bin):
    hexagon = {}
    new_time_bin = {}
    for pair in time_bin.keys():
        if len(pair) == 1:
            hexagons[list(pair)[0]] = time_bin[pair]
        else:
            new_time_bin[pair] = time_bin[pair]
    return new_time_bin, hexagons

In [33]:
time_bins_dist = calc_dist_time_bin(df, matrix)

In [34]:
selected_time_bin = time_bins[5]
time_bin = time_bins_dist[selected_time_bin]
time_bin, hexagons = get_hexagons(time_bin)
threshold = 0.4

In [35]:
time_bin = scale_distances(time_bin)

In [46]:
# function that empty hexagons on a map with only the boarders for the hexagons which hold sampels
def draw_sample_hexagons(hex_dict, m=None, color='grey', zoom_start=1):
    # Create a map if it is not provided
    if m is None:
        m = folium.Map(location=(0.0, 0.0), tiles="Esri worldstreetmap", zoom_start=zoom_start)

    # Plot hexagons
    for hexagon in hex_dict.keys():
        # split the hexagon if it crosses the antimeridian
        parts = split_hexagon_if_needed(hexagon)
        for part in parts:
            polygon = folium.Polygon(
                locations=part,
                weight=1,
                color=color,
                fill_opacity=0.0,
                fill=True
            )
            # add tooltip with internal hexagon distance
            polygon.add_child(folium.Tooltip(f"Internal average sample distance: {hex_dict[hexagon]}"))
            polygon.add_to(m)
    return m


# function that draws hexagons on a map
def draw_hexagons(hexagons, m=None, color='darkgreen', zoom_start=1, value=None, opacity=0.5, imputed=False):
    # Create a map if it is not provided
    if m is None:
        m = folium.Map(location=(0.0, 0.0), tiles="Esri worldstreetmap", zoom_start=zoom_start)

    # Plot hexagons
    for hexagon in hexagons:
        # split the hexagon if it crosses the antimeridian
        parts = split_hexagon_if_needed(hexagon)
        for part in parts:
            polygon = folium.Polygon(
                locations=part,
                weight=1,
                color=None,
                fill_color=color,
                fill_opacity=opacity,
                fill=True
            )
            if value >= 0:
                if imputed:
                    polygon.add_child(folium.Tooltip(f"{value} (Imputed)"))
                else:
                    polygon.add_child(folium.Tooltip(value))
            
            polygon.add_to(m)
    return m

def draw_hexagons_with_values(hex_dict, m=None, zoom_start=1, threshold=0.0, imputed=False):
    hexagons = hex_dict.keys()
    values = hex_dict.values()
    
    # get color gradient
    cmap = get_color_gradient()

    # write the values to the center of each hexagon
    for hexagon, value in zip(hexagons, values):
        if value < threshold:
            continue
        col = mcolors.to_hex(cmap(value))
        m = draw_hexagons([hexagon], m, color=col, zoom_start=zoom_start, value=value, imputed=imputed)
    return m

In [ ]:
lines = get_distance_lines(time_bin)
m = draw_sample_hexagons(hexagons)
m = draw_barriers(lines, m)
m

In [38]:
threshold = 0.4
isolated_hex, barrier_lines, barrier_hex = get_isolated_hex_and_barriers(time_bin, hexagons, threshold, allowed_distance=10)

print(f"Number of isolated hexagons: {len(isolated_hex)}")
print(f"Number of barrier lines: {len(barrier_lines)}")
print(f"Number of barrier hexagons: {len(barrier_hex)}")

Number of isolated hexagons: 12
Number of barrier lines: 50
Number of barrier hexagons: 670


In [39]:
# get the index of the time bin of interest
time_bin_index = time_bins.index(time_bins[5])
# get the closest population for each isolated hexagon
closest_populations, new_isolated_hex = find_closest_population(df, time_bin_index, isolated_hex, matrix, threshold)
print(f"Number of isolated hexagons with no migration: {len(new_isolated_hex)}")

Number of isolated hexagons with no migration: 8


In [42]:
imputed_hex = impute_missing_hexagons(barrier_hex)

In [48]:
#m = draw_hexagons(new_isolated_hex, color = "red", m = m, opacity=0.7, value="No migration")
m = draw_hexagons_with_values(barrier_hex, m)
m = draw_hexagons_with_values(imputed_hex, m, imputed=True)
m = draw_sample_hexagons(hexagons, m)
m = draw_barriers(barrier_lines, m)
m = draw_migration_for_time_bin(closest_populations, m)
m